In [275]:
import pandas as pd
import fitz
path = "C:/DAEN690/1A8.pdf"
pdf = fitz.open(path)
pdf.metadata

{'format': 'PDF 1.6',
 'title': '1A8 Rev 38',
 'author': 'FAA',
 'subject': 'TCDS',
 'keywords': '',
 'creator': 'Acrobat PDFMaker 10.1 for Word',
 'producer': 'Adobe PDF Library 10.0',
 'creationDate': "D:20150831163836-05'00'",
 'modDate': "D:20201217085433-05'00'",
 'trapped': '',
 'encryption': None}

In [276]:
#add page to all words
def add_page(in_put, value):
    inn = []
    out = [[]]
    index = 0
#    v = str(value+1)
    for i in range(0, len(in_put)):
        if(i < len(in_put)-1):
            inn += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            inn.insert( 8, value+1)
            out[index] = inn
            index += 1
            inn = []
            out += [[]]
        else:
            inn += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            inn.insert( 8, value+1)
            out[index] = inn

    return out

#get all text from given pdf file
def get_all_text(data):
    pc = data.page_count
    word_box = []
    for i in range(0,pc):
        ct = data.load_page(i)
        gw = ct.get_text('words')
        # If the df do not need page anymore, #agw = ... and make wor_box += gw
        agw = add_page(gw, i)
        word_box += agw
    
    return word_box

# merge the words by block_no and line_no
def word_line(in_put):
    index = 0
    word_list = []
    output = [[]]
    for i in range(0,len(in_put)-1):
        if(in_put[i][7]+1 == in_put[i+1][7]):
            word_list += [in_put[i][4]]  
        else:
            if(in_put[i+1][7]==0):
                word_list += [in_put[i][4]]
                word_list = [' '.join(word_list)]
                word_list += [in_put[i][5], in_put[i][6], in_put[i][7],in_put[i][8]]
                word_list.insert( 0, in_put[i-in_put[i][7]][0])
                word_list.insert( 1, in_put[i-in_put[i][7]][1])
                word_list.insert( 2, in_put[i][2])
                word_list.insert( 3, in_put[i][3])
            #print(word_list)

            output[index] += word_list
            index += 1
            word_list = []
            output += [[]]
    output.pop(-1)
    return output

# detect the word lines' class (key_term, header, info)
def detect_khi(in_put):
    index = 0
    temp_list= []
    out_put = [[]]
    for i in range(0,len(in_put)):
        ## old one add & (in_put[i][6] == 0) in both if & elif
        if(in_put[i][0] <= 106) & (in_put[i][7] < 5 ):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7], in_put[i][8]]
            temp_list.insert(8, 'key_term')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        elif(in_put[i][0] <= 106) & (in_put[i][4].find('Model') > 0):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7], in_put[i][8]]
            temp_list.insert(8, 'header')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        else:
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7], in_put[i][8]]
            temp_list.insert(8, 'info')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
    out_put.pop(-1)
    #print(out_put)
    return out_put

# use to catch data in a single page
def gp(in_put):
    out = []
    for i in range(0,len(in_put)-8):
        if(in_put[i][8] == 1):
            out += [in_put[i]]
    return out

# merge the lines that have same y0 in same page 

def merge_info_same_y0(df):
    fill_lst = [0,0,0,0,0,0,0,0,0,0]
    df += [[]]
    df[-1] = fill_lst
    num = 0
    index = 0
    lst = []
    out = [[]]
    
    for i in range(0 ,len(df)-1):
        if(df[i][8] == 'info') & (df[i+1][8] == 'info') & (round(df[i][1]) == round(df[i+1][1])):
            lst += [df[i][4]]
            num += 1
        else:
            lst += [df[i][4]]
            lst = ['  '.join(lst)]
            lst += [df[i][5], df[i][6], df[i][7], df[i][8], df[i][9]]
            lst.insert( 0, df[i-num][0])
            lst.insert( 1, df[i][1])
            lst.insert( 2, df[i][2])
            lst.insert( 3, df[i][3])
            
            out[index] = lst
            index += 1
            lst = []
            out += [[]]
            num = 0
    out.pop(-1)
    return out

# merge the lines that have same x0 in same page, and the selected lines must in the right side 
def merge_all_info(df):
    fill_lst = [0,0,0,0,0,0,0,0,0,0]
    df += [[]]
    df[-1] = fill_lst
    num = 0
    index = 0
    lst = []
    out = [[]]
    for i in range(0 ,len(df)-1):
        if(df[i][8] == 'info') & (df[i+1][8] == 'info') & (int(df[i][0]) == int(df[i+1][0])) & (df[i][0] > 106):
            lst += [df[i][4]]
            num += 1
        else:
            lst += [df[i][4]]
            lst = ['  '.join(lst)]
            lst += [df[i][5], df[i][6], df[i][7], df[i][8], df[i][9]]
            lst.insert( 0, df[i-num][0])
            lst.insert( 1, df[i][1])
            lst.insert( 2, df[i][2])
            lst.insert( 3, df[i][3])
            
            out[index] = lst
            index += 1
            lst = []
            out += [[]]
            num = 0
    out.pop(-1)
    return out    


# pre process the data to data frame and ready to convert to excel file
def prep_to_xlsx(df):
    index = 0
    lst = []
    out = [[]]
    for i in range(0, len(df)):
        if(df[i][0] < 106):
            lst = [df[i][4], ' ']
            out[index] = lst
            lst = []
            out += [[]]
            index += 1
        else:
            lst = [' ', df[i][4]]
            out[index] = lst
            lst = []
            out += [[]]
            index += 1
    
    out.pop(-1)
    return out

In [277]:
#get all text [x0, y0, x1, y1, word, block_no, line_no, word_no, page]
all_text = get_all_text(pdf)


# merge text by block_no & line_no
out = word_line(all_text)
#print(out)


#sorted key + lambda 
# ref: https://blog.csdn.net/w417950004/article/details/86253721
# reorder the word lines by coordinates (page > y0 > x0)
pall = sorted(out, key = lambda x:(int(x[8]), float(x[1]), float(x[0])))
#for i in range(0, len(pall)):
#    print(pall[i][8], pall[i][1], pall[i][0], pall[i][4])


# classify the word to (header , key_term, info), 
#the output will be [x0, y0, x1, y1, word, block_no, line_no, word_no, class, page]
khip1 = detect_khi(pall)
#print(khip1)


# Use page1 to test the function
# Be used to select page, if need all df, ignore this 
#p1 = gp(out)
#for i in range(0,len(p1)):
#    print(p1[i][8])

infoSy0 = merge_info_same_y0(khip1)
#for i in range(0, len(infoSy0)):
#    print(infoSy0[i][4])

info = merge_all_info(infoSy0)
#for i in range(0, len(info)):
#    print(info[i][4])



In [278]:
# This is merge info version
px = prep_to_xlsx(info)
#for i in range(0, len(px)):
#print(px)
df = pd.DataFrame(px, columns=['L', 'R'])
df

,L,R
0,,DEPARTMENT OF TRANSPORTATION
1,,FEDERAL AVIATION ADMINISTRATION
2,,1A8
3,,Revision 38
4,,HELIO H-250
...,...,...
467,,Airworthiness Limitations for any mandatory re...
468,NOTE 3.,
469,,Maintenance Manual (Instructions for Continued...
470,NOTE 4.,


In [279]:
#writer=pd.ExcelWriter('C:/DAEN690/1A13_try.xlsx',engine='xlsxwriter')
#df.to_excel(writer, index=False)
#writer.save()

In [280]:
# This is line by line version 
px1 = prep_to_xlsx(infoSy0)
df1 = pd.DataFrame(px1, columns=['L', 'R'])
df1

,L,R
0,,DEPARTMENT OF TRANSPORTATION
1,,FEDERAL AVIATION ADMINISTRATION
2,,1A8
3,,Revision 38
4,,HELIO
...,...,...
767,,"Reissued to Alliance Aircraft Group, LLC Septe..."
768,,"Reissued to Alliance Aircraft Group, LLC dba H..."
769,,"Reissued to Helio Aircraft, LLC July 14, 2005"
770,,"Reissued to Helio Alaska, Inc. March 30, 2020"


In [236]:
# not done yet
#def xlsx(in_put):
#    lst = in_put
#    df = pd.DataFrame(in_put, columns=['L', 'R'])
#    df.to_excel("1A13_P1.xlsx", index = False)

In [238]:
# xlsx(px)